In [25]:
import os
os.environ["GROQ_API_KEY"] = 'Your_API_Key'

In [ ]:
!pip install -q langchain-groq langchain-core requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 4.4 MB/s eta 0:00:00


In [26]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

## Step(1) Create Tool

In [27]:
@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [28]:
print(multiply.invoke({'a':3, 'b':4}))

12


In [29]:
multiply.name

'multiply'

In [30]:
multiply.description

'Given 2 numbers a and b this tool returns their product'

In [31]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

## Step(2) Tool Binding

In [32]:
groq_api_key = os.getenv("GROQ_API_KEY")

In [33]:
llm = ChatGroq(model_name="llama3-8b-8192", api_key=groq_api_key)

In [34]:
llm.invoke('hi')

AIMessage(content="Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 11, 'total_tokens': 36, 'completion_time': 0.020833333, 'prompt_time': 0.002407964, 'queue_time': 0.017646294, 'total_time': 0.023241297}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'finish_reason': 'stop', 'logprobs': None}, id='run-adbec530-c482-4d6c-bf79-24dd513ffe01-0', usage_metadata={'input_tokens': 11, 'output_tokens': 25, 'total_tokens': 36})

In [35]:
llm_with_tools = llm.bind_tools([multiply])

In [36]:
llm_with_tools.invoke('Hi how are you')

AIMessage(content="I'm doing well, thanks for asking!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 1873, 'total_tokens': 1883, 'completion_time': 0.008333333, 'prompt_time': 0.234336033, 'queue_time': -0.280908489, 'total_time': 0.242669366}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-3aeb5a04-e8da-40e2-8f88-1f7b3e08efde-0', usage_metadata={'input_tokens': 1873, 'output_tokens': 10, 'total_tokens': 1883})

In [37]:
query = HumanMessage('can you multiply 3 with 1000')

In [39]:
messages = [query]

In [40]:
result = llm_with_tools.invoke(messages)

In [41]:
messages.append(result)

In [42]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mghw', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 932, 'total_tokens': 1005, 'completion_time': 0.060833333, 'prompt_time': 0.116772195, 'queue_time': 0.022287262000000002, 'total_time': 0.177605528}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-716af4e2-79e8-4f80-8e9a-1df0fa4805a4-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'call_mghw', 'type': 'tool_call'}], usage_metadata={'input_tokens': 932, 'output_tokens': 73, 'total_tokens': 1005})]

In [43]:
tool_result = multiply.invoke(result.tool_calls[0])

In [44]:
tool_result

ToolMessage(content='3000', name='multiply', tool_call_id='call_mghw')

In [45]:
messages.append(tool_result)

In [46]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mghw', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 932, 'total_tokens': 1005, 'completion_time': 0.060833333, 'prompt_time': 0.116772195, 'queue_time': 0.022287262000000002, 'total_time': 0.177605528}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-716af4e2-79e8-4f80-8e9a-1df0fa4805a4-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'call_mghw', 'type': 'tool_call'}], usage_metadata={'input_tokens': 932, 'output_tokens': 73, 'total_tokens': 1005}),
 ToolMessage(content='3000', name='multiply', tool_call_id='call_mghw')]

In [47]:
llm_with_tools.invoke(messages).content

'3000'